# **Multiclass Classification**

In [18]:
import os
import sys
from collections import Counter

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import PIL
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from torch.utils.data import dataloader, random_split
from torchinfo import summary
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

torch.backends.cudnn.deterministic = True

**Project Goal**: To build and train a convolutional neural network, and use it to get predictions for the animal classes.

**Objectives**

- Read in data with multiple classes.
- Normalize data to improve performance.
- Create a CNN that works well with images.
- Train that network to do multiclass classification.
- Reformat the network predictions to complete the [competition](https://www.drivendata.org/competitions/87/competition-image-classification-wildlife-conservation/). - The goal of the competition is to build a model that takes an image and classifies what animal is in it. 